## The Minimum Cost Prison Diet
**Written by: Team Margaret Reid**

## Introduction and Population (Deliverable 1a)

Prison diets are known to be high in carbohydrates and sodium levels with little or no access to fresh produce. Often, this results in adverse health outcomes and can lead to increased spendings for medical needs. The goal of this project is to identify the minimum cost diet that satisfies nutritional needs for U.S. inmate population using available data on county, state, and federal prisons. We will come up with a minimum cost diet for the average prisoner and investigate whether or not state prisons are able to meet the suggested nutritional requirements based on their weekly menus.

### Setup

In [1]:
# Uncomment to run the line below when you are opening this notebook for the first time
%pip install -r requirements.txt --upgrade

  Using cached gspread-6.2.0-py3-none-any.whl.metadata (11 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from Functions import solvercomplete, nutrition, nutrient_search
#for documentation on any function run "function?", ex nutrition?
import numpy as np
import pandas as pd
from eep153_tools.sheets import read_sheets

### Part 1: Setting constraints and general variables

#### Deliverable 1b: Nutrition function to set constraints

In [12]:
age = 42 # choose an appropriate age
sex = "M" # set M for Male and F for Female
activity_level = "Active" # options: Moderately Active, Sedentary, or Active

In [13]:
# time to make minimizing and maximizing constraints
bmin, bmax = nutrition(age, sex, activity_level)
print(f"Current minimizing constraints are: \n \n {bmin}")
print(f"Current maximum constraints are: \n \n {bmax}")

Current minimizing constraints are: 
 
 Nutrient
Energy            2800.0
Protein             56.0
Carbohydrate       130.0
Dietary Fiber       30.8
Linoleic Acid       17.0
Linolenic Acid       1.6
Calcium           1000.0
Iron                 8.0
Magnesium          420.0
Phosphorus         700.0
Potassium         4700.0
Zinc                11.0
Copper               0.9
Selenium            55.0
Vitamin A          900.0
Vitamin E           15.0
Vitamin D           15.0
Vitamin C           90.0
Thiamin              1.2
Riboflavin           1.3
Niacin              16.0
Vitamin B6           1.3
Vitamin B12          2.4
Choline            550.0
Vitamin K          120.0
Folate             400.0
Name: Male_31_50, dtype: float64
Current maximum constraints are: 
 
 Nutrient
Sodium    2300.0
Name: Male_31_50, dtype: float64


/home/jovyan/EEP-153-Project-2/Functions.py:40: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  bmin[0] = active_energy


### Part 2: Determining recipe and nutrition set

**Nutrition set is what constrains the options, since we are using a left join (or merge) on nutrients and recipes. The Nutrition dataset provides nutritional values and the recipe set provides weights**

In [14]:
data_url = "https://docs.google.com/spreadsheets/d/1xqixhrAoDq9rWJf_FC3Y2eXdd010DTLPCS7JJMCfwP8/edit?usp=sharing"
recipes = read_sheets(data_url, sheet="recipes")
nutrients = read_sheets(data_url, sheet="nutrients")

Now, we used a nutrition search function we developed to filter out some ingredients that are not feasible. Note that for the constrained table we set Cut = True to produce a dataframe free of matching terms.

In [15]:
#example use
#nutrient_search('Cream', nutrients).head()
#other example uses
# nutrient_search(['dog', 'cinnamon'], nutrients) 
# -> this would return all entries containing dog or cinnamon
# nutrient_search(['dog', 'cinnamon'], nutrients, cut = True) 
# -> this would return all entries not containing dog or cinammon (acting as a filter)
# nutrient_search('cheese', nutrients)
# -> this would return all entries containing cheese in ingredient descriptions
updated_nutrients = nutrient_search(['Carp', 'Plantain', 'Wheat bran', 'Brazil', 'Soy flour, defatted'], nutrients, cut=True)
updated_nutrients.head()

,ingred_code,Ingredient description,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,Vitamin B12,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc
0,1001,"Butter, salted",2.529,2.587,7.436,21.697,0.961,9.999,19.961,2.728,...,0.17,0.0,0.003,0.0,0.0,2.32,0.0,7.0,15.87,0.09
1,1002,"Butter, whipped, with salt",2.039,2.354,7.515,20.531,1.417,7.649,17.370,2.713,...,0.07,0.0,0.008,0.0,0.0,1.37,0.0,4.6,16.72,0.05
2,1003,"Butter oil, anhydrous",2.495,2.793,10.005,26.166,2.228,12.056,25.026,2.247,...,0.01,0.0,0.001,0.0,0.0,2.80,0.0,8.6,0.24,0.01
3,1004,"Cheese, blue",0.601,0.491,3.301,9.153,0.816,3.235,6.622,0.536,...,1.22,0.0,0.166,0.0,0.5,0.25,0.0,2.4,42.41,2.66
4,1005,"Cheese, brick",0.585,0.482,3.227,8.655,0.817,3.455,7.401,0.491,...,1.26,0.0,0.065,0.0,0.5,0.26,0.0,2.5,41.11,2.60


## Running the optimization: Our Minimum Cost Diet
Here is a call to the optimization function, passing in the updated constraint nutrients dataframe: *solvercomplete(sex, age,recipes, **updated nutrients**, bmin, bmax)*

In [16]:
solvercomplete(sex, age, recipes, updated_nutrients, bmin, bmax)

Cost of diet for M's of age 42 is $2.76 per day.

The diet will consist of (in 100s of grams or milliliters):
Milk, low fat (1%)                        10.17
Egg, yolk only, raw                        0.15
Split peas, from dried, fat added          7.64
Peanuts, unroasted                         0.14
Cereal, rice flakes                        0.18
Cereal, toasted oat                        0.09
Cilantro, raw                              0.14
Corn oil                                   0.81
Fruit juice drink, with high vitamin C     2.80
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               2800.0          2800.0
Protein          102.291294            56.0
Carbohydrate     257.883387           130.0
Dietary Fiber     61.890972            30.8
Linoleic Acid     64.344459            17.0
Linolenic Acid       3.4323             1.6
Calcium         1455.766178          

'2.76'

## Analysis on State Prison Menus

The lists of food items in this section are adapted from state prison weekly menus in California, Pennyslvania, and Florida. Using these food items to calculate a min-cost diet will tell us whether the menus are able to satisfy nutritional needs and if so, whether or not it is at an affordable price.

In [17]:
cal_food_items = [
    "Apple Juice", "Orange Juice", "Oatmeal", "Creamed Beef", "Biscuit", "Diced Potatoes", 
    "Ketchup", "1% Milk", "Coffee", "Pinto Bean", "Gravy", "Milk", "Peanut Butter", "Jelly", 
    "Almond", "Oatmeal Cookie", "Apple", "Orange", "Banana", "Lettuce", "Chicken Vegetable Stew", 
    "Corn Bread", "Turkey Hot Dog", "Vegetable Stew", "Low Fat Yogurt", "Bran Cake", "Bologna", 
    "Mayonnaise", "Mustard", "Whole Wheat Bread", "Corn Chips", "Coleslaw", "Hot Link", 
    "New Orleans Red Bean", "Red Bean", "Rice", "Cake with Icing", "Chocolate Milk", "Cereal", "Egg", 
    "Onions", "Bell Peppers", "Hummus", "Pretzel", "Sunflower Seed", "Bean & Cheese Burrito", 
    "Breakfast Burritos", "Chicken Chorizo", "Taco Sauce", "Tortillas", "Shredded Cheese", 
    "Roasted Chicken Lunchmeat", "Peanut Butter Cracker", "Gold Fish", "Chicken Taco Salad", 
    "Corn Tortilla", "Cheese Pizza", "Italian White Beans", "Pancake", "Syrup", "Beef Ravioli", 
    "Tuscan Style Bean", "Sugar Free Beverage", "Carrots", "Broccoli", "Chicken"
]

In [18]:
cal_nutrients = nutrient_search(cal_food_items, nutrients, cut = False)
solvercomplete(sex, age, recipes, cal_nutrients, bmin, bmax)

Cost of diet for M's of age 42 is $3.49 per day.

The diet will consist of (in 100s of grams or milliliters):
Milk, low fat (1%)                             7.56
Egg, yolk only, raw                            0.31
Peanut butter, reduced fat                     1.30
Chinese pancake                                4.09
Cereal, toasted oat                            0.60
Banana, raw                                    5.25
Mustard greens, fresh, cooked, no added fat    0.30
Mayonnaise, regular                            0.22
Fruit juice drink, added calcium (Sunny D)     0.43
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               2800.0          2800.0
Protein           93.681575            56.0
Carbohydrate     442.529278           130.0
Dietary Fiber          30.8            30.8
Linoleic Acid     23.499604            17.0
Linolenic Acid          1.6             1.6
C

'3.49'

In [19]:
pen_food_items = [
    "Grit", "Egg", "Toast", "Margarine", "Milk", "Coffee", "Sugar", 
    "Apple", "Orange", "Banana", "Meatloaf Patty", "Soy Loaf", "Gravy", "Whipped Potato", 
    "Wax Bean", "Bread", "Grilled Cheese Sandwich", "Peanut Butter", "Jelly", "Tomato Soup", 
    "Pasta", "Oatmeal", "Breakfast Pastry", "Hamburger", "Bean Burger", "Potato", "Corn", 
    "Ketchup", "Mustard", "Mayonnaise", "Coleslaw", "Rice", "Cornbread", "Frankfurters", 
    "Macaroni Salad", "Baked Bean", "Broccoli", "Jello", "Hard Cooked Eggs", 
    "Poultry Cacciatore", "Soy Cacciatore", "Pasta and Bean Casserole", "Lettuce", 
    "Salad Dressing", "Cream of Wheat", "Breaded Chicken Patty", "Pinto Beans", 
    "Spinach", "Noodle", "Carrot", "Meat Sauce Pasta Casserole", 
    "Soy Pasta Sauce Pasta Casserole", "Breaded Fish", "Creamed Soy Crumble", 
    "Creamed Beef", "Bean Paste", "Cocktail Sauce", "Tartar", "Vegetable Soup", 
    "Pancake", "Syrup", "Pudding", "Cheese Steak", "Tofu Oat Burger", "Flour Tortilla"
]

In [20]:
pen_nutrients = nutrient_search(pen_food_items, nutrients, cut = False)
solvercomplete(sex, 42, recipes, pen_nutrients, bmin, bmax)

Cost of diet for M's of age 42 is $2.97 per day.

The diet will consist of (in 100s of grams or milliliters):
Hot chocolate / Cocoa, made with no sugar added dry mix and low fat milk    5.52
Egg, yolk only, raw                                                         0.37
Tortilla, corn                                                              2.33
Pasta, whole grain, cooked                                                  0.48
Rice, white, cooked, made with margarine                                    4.54
Cereal, rice flakes                                                         0.05
Potato, roasted, from fresh, peel eaten, no added fat                       3.32
Mustard greens, fresh, cooked, no added fat                                 0.79
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               2800.0          2800.0
Protein           64.514205            56

'2.97'

In [21]:
flo_food_items = [
    "Pancake", "Fresh Fruit", "Syrup", "Biscuit", "Jelly", "Hashbrown Potato", 
    "Sugar Packet", "Meatloaf Patty", "Mashed Potato", "Cabbage", "Sweet Pea", 
    "Dried Beans", "Meatball", "Italian Red Sauce", "Sausage", "Buttery Oatmeal", 
    "Potato Salad", "Fudge Brownies", "Baked Bean", "Rice", "Green Beans", 
    "Broccoli", "Cookie", "Tortilla", "Lettuce", "Pasta Salad", "Herbed Chicken Leg", 
    "Bran Flaked Cereal", "Carrot", "Scrambled Egg", "Vegetarian Cheese", "Oatmeal", 
    "Toast", "Rice", "Pasta", "Bread", "Orange", "Banana", "Apple", "Margarine", 
    "Tea", "Coffee", "Corn", "Sloppy Joe", "Taco Skillet", "Dressing", "Gravy", 
    "Grits", "Turkey", "Country Patty", "Pasta", "Buffalo Chicken Patty", "Cheese", 
    "Coleslaw", "Southern BBQ", "Bran Muffin", "Blueberry Muffin", "Potato"
]

In [22]:
flo_nutrients = nutrient_search(flo_food_items, nutrients, cut = False)
solvercomplete(sex, age,recipes, flo_nutrients, bmin, bmax)

Cost of diet for M's of age 42 is $4.31 per day.

The diet will consist of (in 100s of grams or milliliters):
Liverwurst                                               1.56
Tortilla, corn                                           0.50
Pasta, whole grain, cooked                               1.64
Rice, white, cooked, made with margarine                 2.04
Cereal, rice flakes                                      0.21
Oatmeal cereal, baby food, dry, instant                  0.68
Potato, roasted, from fresh, peel eaten, no added fat    6.81
Cabbage, green, raw                                      0.30
Corn oil                                                 0.37
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               2800.0          2800.0
Protein           64.887773            56.0
Carbohydrate     325.157438           130.0
Dietary Fiber          30.8            30.8

'4.31'

In [23]:
# This returns a min-cost diet by using food items from across the 3 state prison menus.

combined_nutrients = nutrient_search(cal_food_items + pen_food_items + flo_food_items, nutrients)
solvercomplete(sex, age,recipes, combined_nutrients, bmin, bmax)

Cost of diet for M's of age 42 is $2.90 per day.

The diet will consist of (in 100s of grams or milliliters):
Hot chocolate / Cocoa, made with no sugar added dry mix and low fat milk    3.43
Egg, yolk only, raw                                                         0.38
Peanut butter, lower sugar                                                  0.62
Rice, white, cooked, made with margarine                                    5.11
Cereal (Kellogg's All-Bran)                                                 0.41
Potato, roasted, from fresh, peel eaten, no added fat                       3.98
Mustard greens, fresh, cooked, no added fat                                 0.62
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               2800.0          2800.0
Protein           62.021397            56.0
Carbohydrate     252.187393           130.0
Dietary Fiber          30.8       

'2.90'